#### **GIS-Based Estimation of Crop Coefficients ($K_c$) using Sentinel-2 and MODIS Data Fusion**
#### **Purpose:** This project implements a multi-sensor remote sensing pipeline to estimate high-resolution Crop Coefficients ($K_c$) for the Bacita Sugar Estate. By fusing coarse-resolution Evapotranspiration data (MODIS) with fine-resolution vegetation indices (Sentinel-2) using a calibrated linear regression model, the study downscales global water demand data to field-level precision. The output visualizes irrigation priority zones and quantifies water stress distributions to support data-driven command area management.

##### **Author:** Bello Oluwatobi

##### **Last Updated:** October 6, 2025

### #1 Importing libraries

In [ ]:
# importing necessary libraries
import ee
import geemap
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import time

### #1 Initializing the Google Earth Engine

In [ ]:
# setting the google earth engine project
# initializing only once when the cell is first run
try:
    ee.Initialize(project='YOUR_PROJECT_ID')
except:
    ee.Authenticate()
    ee.Initialize(project='YOUR_PROJECT_ID')

### #2 Setting the Study Region: Bacita Sugar Estate

In [ ]:
# setting the region and point for Bacita Sugar Estate
bacita_point = ee.Geometry.Point([4.965, 9.075])
bacita_roi = ee.Geometry.Rectangle([4.92, 9.05, 5.02, 9.12])

### #3 Extracting the MODIS and Setinel-2 Datasets and merging them

In [ ]:
# specifying the datasets (using MODIS 6.1 and Sentinel-2)
modis_col = ee.ImageCollection('MODIS/061/MOD16A2').filterDate('2023-01-01', '2025-01-01').filterBounds(bacita_point)
s2_col = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED').filterBounds(bacita_point)

In [ ]:
# defining functions to extract the datasets
def extract_matched_data(image):
    date = image.date()

    # extracting MODIS ET and PET values explicitly
    modis_stats = image.select(['ET', 'PET']).reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=bacita_point,
        scale=500
    )

    # finding clear sentinel images (+/- 5 days)
    s2_match_col = s2_col.filterDate(date.advance(-5, 'day'), date.advance(5, 'day')) \
                         .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))

    # checking if a matching Sentinel image exists
    count = s2_match_col.size()

    # writing the conditional logic for NDVI
    ndvi_val = ee.Algorithms.If(count.gt(0),
                                s2_match_col.median().normalizedDifference(['B8', 'B4']).reduceRegion(ee.Reducer.mean(), bacita_point, 20).get('nd'),
                                -999)

    return ee.Feature(None, {
        'Date': date.format('YYYY-MM-dd'),
        'ET': modis_stats.get('ET'),
        'PET': modis_stats.get('PET'),
        'NDVI': ndvi_val
    })

In [ ]:
# running the extraction and converting to DataFrame
print("Extracting multi-sensor data for Bacita...")
extracted_features = modis_col.map(extract_matched_data)
df = geemap.ee_to_df(ee.FeatureCollection(extracted_features))

In [ ]:
# cleaning the data
# converting to numeric to handle any potential strings or objects
df['ET'] = pd.to_numeric(df['ET'], errors='coerce')
df['PET'] = pd.to_numeric(df['PET'], errors='coerce')
df['NDVI'] = pd.to_numeric(df['NDVI'], errors='coerce')

# filtering the valid rows
df = df.dropna(subset=['ET', 'PET', 'NDVI'])
df = df[df['NDVI'] != -999]
df = df[(df['ET'] < 30000) & (df['PET'] < 30000)] # removing fill values

# calculating the Crop Coefficient (Kc)
df['Kc_Observed'] = (df['ET'] * 0.1) / (df['PET'] * 0.1)

# saving the data as csv file
df.to_csv('../data/bacita_true_field_data.csv', index=False)

print(f"{len(df)} data points extracted.")
print(df.head())

### #4 Running the Simple Linear Regression

In [ ]:
# loading the dataset
data = pd.read_csv('../data/bacita_true_field_data.csv')

In [ ]:
# executing the Machine Learning phase - Simple Linear Regression
X = data[['NDVI']]
y = data['Kc_Observed']
model = LinearRegression().fit(X, y)
slope, intercept = model.coef_[0], model.intercept_
r_squared = r2_score(y, model.predict(X))

In [ ]:
# creating the analytics plot
plt.style.use('seaborn-v0_8-whitegrid')
plt.figure(figsize=(12, 7))

# generating scatter plot of real observations
sns.scatterplot(x='NDVI', y='Kc_Observed', data=data, s=100, color='#2c3e50', alpha=0.6, label='Satellite Observations')

# setting the regression Line
plt.plot(data['NDVI'], model.predict(X), color='#e74c3c', linewidth=3, label=f'Model: Kc = {slope:.2f}*NDVI + ({intercept:.2f})')

plt.title('Sugarcane Water Demand Analysis: NDVI vs. $K_c$ (Bacita Estate)', fontsize=16, fontweight='bold')
plt.xlabel('Vegetation Vigor (Sentinel-2 NDVI)', fontsize=13)
plt.ylabel('Water Demand Proxy (MODIS Kc)', fontsize=13)
plt.text(0.1, 0.9, f'R-Squared: {r_squared:.2f}', transform=plt.gca().transAxes, fontsize=14, color='red', fontweight='bold')
plt.legend(frameon=True, fontsize=12)
plt.savefig('../results/Bacita_ML_Regression.png', dpi=300)
plt.show()

### #5 Map Initialization and Rendering

In [ ]:
# initializing the map
Map = geemap.Map()
Map.centerObject(bacita_roi, 13)

# getting the high-resolution satellite image data
image = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
    .filterBounds(bacita_roi) \
    .filterDate('2024-12-01', '2025-02-28') \
    .sort('CLOUDY_PIXEL_PERCENTAGE').first().clip(bacita_roi)

# applying the model to the data
ndvi = image.normalizedDifference(['B8', 'B4'])
kc_map = ndvi.multiply(slope).add(intercept).rename('Kc')

# setting the water demand scheme
vis_params = {
    'min': 0.3,
    'max': 1.1,
    'palette': ['#d3d3d3', '#ff0000', '#d9ef8b', '#a6d96a', '#1a9850']
}

Map.addLayer(image, {'bands': ['B4', 'B3', 'B2'], 'max': 3000}, 'Natural Color Base')
Map.addLayer(kc_map, vis_params, 'Water Demand (Kc) Model')

# setting the legend content
legend_dict = {
    'Critical/Bare Soil (Kc < 0.4)': '#d3d3d3',
    'Low Demand/Fallow (0.4-0.6)': '#ff0000',
    'Moderate/Emerging (0.6-0.8)': '#d9ef8b',
    'High Demand/Healthy (0.8-1.0)': '#a6d96a',
    'Peak Demand/Dense (> 1.0)': '#1a9850'
}

Map.add_legend(title="Sugarcane Water Demand (Kc)", legend_dict=legend_dict, position='bottomright')

print("Rendering map... please wait 10 seconds for layers to load.")
time.sleep(10) # allowing time to buffer the graphics

# saving the HTML file
Map.save("../results/Bacita_Final_Project.html")


# displaying the map
Map

In [ ]:
# defining the color palette
colors = ['#d3d3d3', '#ff0000', '#d9ef8b', '#a6d96a', '#1a9850']
cmap = mcolors.LinearSegmentedColormap.from_list("Kc_Spectral", colors)

# converting the Earth Engine image to a numpy array for local plotting
rgb_img = geemap.ee_to_numpy(image.select(['B4', 'B3', 'B2']), region=bacita_roi, scale=10)
kc_array = geemap.ee_to_numpy(kc_map, region=bacita_roi, scale=10)

# creating the subplots
fig, ax = plt.subplots(1, 2, figsize=(20, 10))

# setting left plot: natural color reference
ax[0].imshow(rgb_img / 3000) # normalizing Sentinel-2 reflectance
ax[0].set_title('Sentinel-2 Natural Color (Baseline)', fontsize=15, fontweight='bold')
ax[0].axis('off')

# setting right plot: water demand map
im = ax[1].imshow(kc_array, cmap=cmap, vmin=0.3, vmax=1.1)
ax[1].set_title('Modeled Sugarcane Water Demand (Kc)', fontsize=15, fontweight='bold')
ax[1].axis('off')

# adding colorbar
cbar = fig.colorbar(im, ax=ax[1], orientation='vertical', pad=0.02, aspect=30)
cbar.set_label('Crop Coefficient (Kc)', fontsize=12, fontweight='bold')
cbar.ax.set_yticklabels(['0.3 (Soil)', '0.5', '0.7', '0.9', '1.1 (Peak)'])

plt.tight_layout()
plt.savefig('../results/Bacita_Final_GIS_Plot.png', dpi=300, bbox_inches='tight')
plt.show()

### #6 Plotting Histogram of Crop Coefficient

In [ ]:
# flattening the 2D array and removing invalid pixels
kc_values = kc_array.flatten()
kc_values = kc_values[~np.isnan(kc_values)] # removing NaNs
kc_values = kc_values[(kc_values > 0.2) & (kc_values < 1.2)] # focusing on realistic range

# plotting the histogram
plt.figure(figsize=(12, 7))
n, bins, patches = plt.hist(kc_values, bins=50, color='#a6d96a', edgecolor='black', alpha=0.7)

# adding the legend
plt.axvspan(0.2, 0.4, color='#808080', alpha=0.3, label='Fallow / Bare Soil')
plt.axvspan(0.4, 0.6, color='#fdae61', alpha=0.3, label='Low Density Growth')
plt.axvspan(0.6, 0.8, color='#d9ef8b', alpha=0.3, label='Maturing Sugarcane')
plt.axvspan(0.8, 1.0, color='#a6d96a', alpha=0.3, label='High Transpiration')
plt.axvspan(1.0, 1.2, color='#1a9850', alpha=0.3, label='Peak Water Demand')

# setting titles and labels
plt.title('Distribution of Sugarcane Water Demand ($K_c$)', fontsize=15, fontweight='bold')
plt.xlabel('Modeled Crop Coefficient ($K_c$)', fontsize=13)
plt.ylabel('Frequency (Number of 10m Pixels)', fontsize=13)
plt.legend(loc='best', frameon=True, facecolor='white')
plt.grid(axis='y', linestyle='--', alpha=0.6)

plt.savefig('../results/Bacita_Kc_Histogram.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# calculating field dimensions
total_area_ha = (len(kc_values) * 100) / 10000 # Each pixel is 10x10m = 100m2
peak_demand_ha = (len(kc_values[kc_values > 0.8]) * 100) / 10000
low_demand_ha = (len(kc_values[(kc_values >= 0.4) & (kc_values <= 0.6)])) / 10000
print(f"Total Analyzed Area: {total_area_ha:.2f} Hectares")
print(f"Area in High/Peak Demand (>0.8 Kc): {peak_demand_ha:.2f} Hectares ({(peak_demand_ha/total_area_ha)*100:.2f}%)")
print(f"Area in Low Demand (0.4-0.6 Kc): {low_demand_ha:.2f} Hectares ({(low_demand_ha/total_area_ha)*100:.2f}%)")